<a href="https://colab.research.google.com/github/J1116/physics/blob/main/Bayesian_optimization220711.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import plotly.graph_objects as go
from tqdm import tqdm
# from scipy.stats import qmc

In [ ]:
class bayesian_opt():
    def __init__(self, bound, kernel="typical", param = None, ac = "MI", beta = 1.0, prior_dist = False, prior_x = None, prior_y = None, num_sample = 10000, delta = 0.01):
      self.bound = bound
      self.kernel = kernel
      self.param = param
      self.ac = ac
      self.beta = beta
      if prior_dist == True:
        self.V = prior_x
        self.V_nor = (self.V - self.bound[:,1]) / (self.bound[:,0] - self.bound[:,1])
        self.y = prior_y
      else:
        self.V_nor = np.random.rand(self.bound.shape[0])
        self.V = (self.bound[:,0] - self.bound[:,1]) * self.V_nor + self.bound[:,1]
        self.V = self.V.reshape(1,-1)
        self.V_nor = self.V_nor.reshape(1,-1)
        self.y = np.array([])
      self.num_sample = num_sample
      self.sigma = np.zeros([])
      self.m = np.zeros([])
      self.delta = delta
      # for i in range(self.bound.shape[0]):
      #   self.x_sample[i] = np.linspace(self.bound[i,1], self.bound[i,0], self.split_num)

    def gram_matrix(self):
      if self.kernel == "typical":
        self.G = np.zeros((self.V_nor.shape[0], self.V_nor.shape[0]))
        for i in range(self.V_nor.shape[0]):
          for j in range(self.V_nor.shape[0]):
            self.G[i,j] = self.typical_kernel(self.V_nor[i,:], self.V_nor[j,:], self.param)
        self.G_inv = np.linalg.pinv(self.G + np.eye(self.G.shape[0]) / self.beta)
      elif self.kernel == "gauss":
        self.G = self.gauss_kernel(self.V_nor, self.param["gamma"])
        self.G_inv = np.linalg.pinv(self.G + np.eye(self.G.shape[0]) / self.beta)
      elif self.kernel == "matern3":
        self.G = np.zeros((self.V_nor.shape[0], self.V_nor.shape[0]))
        for i in range(self.V_nor.shape[0]):
          for j in range(self.V_nor.shape[0]):
            self.G[i,j] = self.matern3(self.V_nor[i,:], self.V_nor[j,:], self.param)
        self.G_inv = np.linalg.pinv(self.G + np.eye(self.G.shape[0]) / self.beta)

    def kernel_func(self, x1, x2):
      if self.kernel == "typical":
        return self.typical_kernel(x1, x2, self.param)
      elif self.kernel == "matern3":
        return self.matern3(x1, x2, self.param)

    def gaussian_process(self, x):
      k = np.zeros((x.shape[1], self.V_nor.shape[0]))
      for j in range(self.V_nor.shape[0]):
        for i in range(x.shape[1]):
          k[i,j] = self.kernel_func(self.V_nor[j,:], x[:,i])
      m = np.dot(k, np.dot(self.G_inv, self.y))
      c = np.zeros(x.shape[1])
      sigma = np.zeros(x.shape[1])
      for i in range(x.shape[1]):
        c[i] = self.kernel_func(x[:,i], x[:,i]) + 1/self.beta
        sigma[i] = c[i] - np.dot(k[i,:], np.dot(self.G_inv, k[i,:].T))
      return m, sigma

    def acquisition_function(self, m, sigma):
      if self.ac == "EI":
        f_max = np.amax(self.y)
        xi = 0.01
        self.z = (m - f_max - xi) / sigma
        EI = np.zeros(len(m))
        for i in range(len(m)):
          EI[i] = (m[i] - f_max - xi) * norm.cdf(self.z[i]) + sigma[i] * norm.pdf(self.z[i])
        return EI
      if self.ac == "UCB":
        num_data = self.V_nor.shape[0]
        return self.UCB(m, sigma, num_data)
      if self.ac == "MI":
        sigma_sum = np.sum(self.sigma)
        return self.MI(m, sigma, sigma_sum, self.delta)

    def fit(self, y_new):
      # bound is (num_data, 2) matrix which explain the range of each parameters.
      # self.bound = bound
      self.y = np.append(self.y, y_new)
      self.gram_matrix()
      self.x_sample = self.Latin_Hyper_Cube(self.num_sample, self.bound).T
      m, sigma = self.gaussian_process(self.x_sample)
      ac_val = self.acquisition_function(m, sigma)
      index_max = np.argmax(ac_val)
      self.x_next_nor = self.x_sample[:,index_max].reshape(1,-1)
      self.m = np.append(self.m, m[index_max])
      self.sigma = np.append(self.sigma, sigma[index_max])
      self.x_next = (self.bound[:,0] - self.bound[:,1]) * self.x_sample[:,np.argmax(ac_val)] + self.bound[:,1]
      self.x_next = self.x_next.reshape(1,-1)
      self.V_nor = np.concatenate([self.V_nor, self.x_next_nor])
      self.V = np.concatenate([self.V, self.x_next])
      return self.x_next

    def pred(self, y_new, N):
      self.y = np.append(self.y, y_new)
      self.gram_matrix()
      self.x_sample = self.Latin_Hyper_Cube(N, self.bound).T
      m, sigma = self.gaussian_process(self.x_sample)
      # ac_val = self.acquisition_function(m, sigma)
      index_max = np.argmax(m)
      self.x_next_nor = self.x_sample[:,index_max].reshape(1,-1)
      self.m = np.append(self.m, m[index_max])
      self.sigma = np.append(self.sigma, sigma[index_max])
      self.x_next = (self.bound[:,0] - self.bound[:,1]) * self.x_sample[:,np.argmax(m)] + self.bound[:,1]
      self.x_next = self.x_next.reshape(1,-1)
      # self.V_nor = np.concatenate([self.V_nor, self.x_next_nor])
      # self.V = np.concatenate([self.V, self.x_next])
      return self.x_next
    
    @staticmethod
    def typical_kernel(x1, x2, param):
      return param["a0"] * np.exp(-param["a1"] * (np.linalg.norm(x1 - x2, ord = 2) ** 2)) #+ param["a2"] + param["a3"] * np.dot(x1,x2)
    
    @staticmethod
    def matern3(x1, x2, param):
      return (1 + np.sqrt(3) * np.linalg.norm(x1 - x2, ord = 2) / param["a0"]) * np.exp(-np.sqrt(3) * np.linalg.norm(x1 - x2, ord = 2) / param["a0"])

    @staticmethod
    def gauss_kernel(V,gamma):
      G = np.zeros((V.shape[0], V.shape[0]))
      for i in range(V.shape[0]):
        for j in range(V.shape[0]):
          G[i,j] = np.exp(-gamma * (np.linalg.norm(V[i,:] - V[j,:], ord = 2) ** 2))
      return G
    
    @staticmethod
    def UCB(m, sigma, n):
      ac_val = m + np.sqrt(np.log(n) / n) * sigma
      return ac_val

    @staticmethod
    def MI(m, sigma, sigma_sum, delta):
      ac_val = m + np.sqrt(np.log(2 / delta)) * (np.sqrt(sigma + sigma_sum) - np.sqrt(sigma_sum))
      return ac_val

    # staticmethod
    # def Latin_Hyper_Cube(num_sample, bound):
    #   n = bound.shape[0]
    #   M = num_sample
    #   f = lambda x, d: (bound[d,0] - bound[d,1]) * x + bound[d,1]
    #   g = lambda x: (x - rng.uniform()) / M
    #   rng = np.random.RandomState()
    #   rnd_grid = np.array([rng.permutation(list(range(1, M + 1))) for _ in range(n)])
    #   lhs = np.array([[f(g(rnd_grid[d][m]), d) for d in range(n)] for m in range(M)])
    #   return lhs

    @staticmethod
    def Latin_Hyper_Cube(num_sample, bound):
      n = bound.shape[0]
      M = num_sample
      rng = np.random.RandomState()
      rnd_grid = np.array([rng.permutation(list(range(1, M + 1))) for _ in range(n)])
      lhs = (rnd_grid - np.random.rand(n,M)) / M
      return lhs.T

In [ ]:
def Latin_Hyper_Cube_rev(num_sample, bound):
  n = bound.shape[0]
  M = num_sample
  # f = lambda x, d: (bound[d,0] - bound[d,1]) * x + bound[d,1]
  # g = lambda x: (x - rng.uniform()) / M
  rng = np.random.RandomState()
  rnd_grid = np.array([rng.permutation(list(range(1, M + 1))) for _ in range(n)])
  # lhs = np.array([[g(rnd_grid[d][m]) for d in range(n)] for m in range(M)])
  lhs = (rnd_grid - np.random.rand(n,M)) / M
  return lhs

In [ ]:
num_sample = 1000
n = 100
bound = np.array([[1,-1]] * n)
n = bound.shape[0]
M = num_sample
# f = lambda x, d: (bound[d,0] - bound[d,1]) * x + bound[d,1]
# g = lambda x: (x - rng.uniform()) / M
rng = np.random.RandomState()
rnd_grid = np.array([rng.permutation(list(range(1, M + 1))) for _ in range(n)])
# lhs = np.array([[g(rnd_grid[d][m]) for d in range(n)] for m in range(M)])
lhs = (rnd_grid - np.random.rand(n,M)) / M
lhs = lhs.T

In [ ]:
lhs.shape

(1000, 100)

In [ ]:
param = {"a0" : 1.0, "a1" : 200.0, "a2" : 0, "a3" : 0}
n = 3
x1 = np.random.rand(n)
x2 = np.random.rand(n)
ker = bayesian_opt.typical_kernel(x1, x2, param)
ker

0.013401365859443766

In [ ]:
import time
param = {"a0" : 1.0, "a1" : 200.0, "a2" : 0, "a3" : 0}
n_list = np.array([3,10,100,200,300,400,500,600,700,800,900,1000,1500,2000,3000])
time_list = np.zeros(len(n_list))
rep_num = 1000
num_sample = 1000
for i in range(len(n_list)):
  time_tot = 0
  for j in range(rep_num):
    n = n_list[i]
    # x1 = np.random.rand(n)
    # x2 = np.random.rand(n)
    time_sta = time.perf_counter()
    ker = bayesian_opt.Latin_Hyper_Cube(num_sample, bound)
    time_end = time.perf_counter()
    time_tot += time_end - time_sta
  time_list[i] = time_tot / rep_num

time_list

array([1.3082942e-05, 9.2553400e-06, 1.0156429e-05, 9.7562400e-06,
       1.0004114e-05, 9.7814230e-06, 9.8926350e-06, 1.0092195e-05,
       1.0668767e-05, 1.4795504e-05, 1.2745653e-05, 1.0828025e-05,
       1.2116563e-05, 1.1023581e-05, 1.2033944e-05])

In [ ]:
import time

# sampler = qmc.LatinHypercube(d=2)
# sample = sampler.random(n=5)

param = {"a0" : 1.0, "a1" : 200.0, "a2" : 0, "a3" : 0}
n_list = np.array([3,10,100,200,300,400,500,600,700,800,900,1000,1500,2000,3000])
time_list = np.zeros(len(n_list))
rep_num = 1
num_sample = 10000
for i in range(len(n_list)):
  time_tot = 0
  for j in range(rep_num):
    n = n_list[i]
    # bound = np.array([[1,-1]] * n)
    # x1 = np.random.rand(n)
    # x2 = np.random.rand(n)
    time_sta = time.perf_counter()
    sampler = qmc.LatinHypercube(d=n)
    sample = sampler.random(n=num_sample)
    time_end = time.perf_counter()
    time_tot += time_end - time_sta
  time_list[i] = time_tot / rep_num

time_list

ImportError: ignored

In [ ]:
fig = go.Figure()
# x_array = np.array([np.linspace(20,-20,1000),np.linspace(20,-20,1000)])
fig.add_trace(go.Scatter(x=n_list, y=time_list,
                    mode='markers',
                    name='time_comparing',
                    # marker = dict(
                    #     color = Y[:pos_data_num])
                    ))
fig.show()

In [ ]:
fig = go.Figure()
x_array = np.array([np.linspace(20,-20,1000),np.linspace(20,-20,1000)])
fig.add_trace(go.Scatter(x=x_array[0], y=test_func(x_array),
                    mode='lines',
                    name='test_func',
                    # marker = dict(
                    #     color = Y[:pos_data_num])
                    ))
fig.show()

NameError: ignored

### latest

In [ ]:
def test_func(x):
  return (40*np.sin(x[0]/1.0)-np.power(0.3*(x[0]+6.0),2)-np.power(0.2*(x[0]-4.0),2)-1.0*np.abs(x[0]+2.0))/40

In [ ]:
def test_func(x):
  return np.exp(-1 * x[0]**2)*np.exp(-1 * x[1]**2)*np.cos(4*np.pi*x[0])*np.cos(4*np.pi*x[1])

In [ ]:
def test_func(x):
  return np.exp(-1 * x[0]**2)*np.exp(-1 * x[1]**2)*np.exp(-1 * x[2]**2)*np.exp(-1 * x[3]**2)*np.exp(-1 * x[4]**2)

In [ ]:
def test_func(x):
  return 40.0*np.exp(-((x[0]-1)**2)/100)*np.exp(-((x[1]-1)**2)/100)*np.exp(-((x[2]-1)**2)/100)*np.exp(-((x[3]-1)**2)/100)*np.exp(-((x[4]-1)**2)/100)

In [ ]:
prior_x = np.array([[0.2,0.2,0.2,0.2,0.2],[-0.2,-0.2,-0.2,-0.2,-0.2],[0.5,0.5,0.5,0.5,0.5]])
prior_y = test_func(prior_x.T)
prior_y[:-1]

array([0.81873075, 0.81873075])

In [ ]:
stop_iter = 30
bound = np.array([[20, -20]])
param = {"a0" : 1.0, "a1" : 200.0, "a2" : 0, "a3" : 0}
# param = {"a0" : 0.5}
model = bayesian_opt(bound = bound, param = param, ac = "MI", beta = 100, kernel = "typical",delta = 1e-6)
x_new = model.V[0]
y_new = np.array([test_func(x_new)])
for i in tqdm(range(stop_iter)):
  x_new = model.fit(y_new)
  y_new = np.array([test_func(x_new[0])])

100%|██████████| 30/30 [01:13<00:00,  2.45s/it]


### true latest ver

In [ ]:
stop_iter = 10
bound = np.array([[1, -1],[1, -1]])
param = {"a0" : 1.0, "a1" : 100.0, "a2" : 0, "a3" : 0}
model = bayesian_opt(bound = bound, param = param, ac = "MI", beta = 100, delta = 1e-6)
x_new = model.V[0]
y_new = np.array([test_func(x_new)])
for i in tqdm(range(stop_iter)):
  x_new = model.fit(y_new)
  y_new = np.array([test_func(x_new[0])])

100%|██████████| 10/10 [00:12<00:00,  1.24s/it]


In [ ]:
stop_iter = 10
bound = np.array([[1, -1]] * 3000)
param = {"a0" : 1.0, "a1" : 100.0, "a2" : 0, "a3" : 0}
model = bayesian_opt(bound = bound, param = param, ac = "MI", beta = 100, delta = 1e-6)
x_new = model.V[0]
y_new = np.array([test_func(x_new[:2])])
for i in tqdm(range(stop_iter)):
  x_new = model.fit(y_new)
  y_new = np.array([test_func(x_new[0][:2])])

100%|██████████| 10/10 [01:10<00:00,  7.07s/it]


In [ ]:
x_new

array([[-0.99811099, -0.24667793]])

In [ ]:
model.V[:,0]

array([-0.97873796, -0.91637506, -0.64700978, -0.60824603, -0.94319756,
       -0.32273016, -0.40123342, -0.08853965,  0.01971118,  0.21054032,
        0.24678006])

In [ ]:
df = pd.DataFrame({'rot_angle1': model.V[:-1,0],
                    'rot_angle2': model.V[:-1,1],
                    'y': model.y})

In [ ]:
df

,rot_angle1,rot_angle2,y
0,-0.978738,-0.926557,0.094650
1,-0.916375,-0.547366,0.131653
2,-0.647010,-0.765205,0.098219
3,-0.608246,-0.410755,0.052967
4,-0.943198,-0.214022,-0.266818
5,-0.322730,-0.640183,0.069258
6,-0.401233,-0.986350,0.102639
7,-0.088540,-0.912051,0.085756
8,0.019711,-0.587600,0.310738
9,0.210540,-0.735187,0.481625


In [ ]:
df.to_csv("test.csv")

In [ ]:
model.y

array([-0.04677852,  0.00246639, -0.03339836,  0.18900236,  0.00733341,
       -0.39216338, -0.1027422 , -0.33875157, -0.68865796, -0.14740785,
        0.15749309,  0.18548315, -0.27989529,  0.28581903, -0.34759815,
       -0.15495724, -0.1577428 ,  0.07078223, -0.29737641, -0.66602262,
       -0.13384972,  0.01656656,  0.05910059,  0.73104567,  0.63958734,
        0.5971096 ,  0.58223411,  0.77797442,  0.78024072,  0.77490158,
        0.77715566,  0.76771789,  0.77788972,  0.77595109,  0.77281054,
        0.76853147,  0.74796157,  0.77896468,  0.77949943,  0.77824383,
        0.76224227])

In [ ]:
best_val = model.pred(y_new, 100000)
best_val

array([[-0.00418716, -0.50070592]])

In [ ]:
stop_iter = 100
bound = np.array([[1, -1],[1, -1],[1, -1],[1, -1],[1, -1]])
param = {"a0" : 1.0, "a1" : 5.0, "a2" : 0, "a3" : 0}
model = bayesian_opt(bound = bound, param = param, ac = "MI", beta = 100, delta = 1e-6, prior_dist = True, prior_x = prior_x, prior_y = prior_y[:-1])
# x_new = model.V[0]
# y_new = np.array([test_func(x_new)])
y_new = np.array([prior_y[-1]])
for i in tqdm(range(stop_iter)):
  x_new = model.fit(y_new)
  y_new = np.array([test_func(x_new[0])])

100%|██████████| 100/100 [12:44<00:00,  7.64s/it]


### blank

In [ ]:
stop_iter = 100
bound = np.array([[20, -20],[20, -20],[20, -20],[20, -20],[20, -20]])
param = {"a0" : 200, "a1" : 0.05, "a2" : 0, "a3" : 0}
model = bayesian_opt(bound = bound, param = param, ac = "UCB")
x_new = model.V[0]
y_new = np.array([test_func(x_new)])
for i in tqdm(range(stop_iter)):
  x_new = model.fit(y_new)
  y_new = np.array([test_func(x_new[0])])

100%|██████████| 100/100 [11:50<00:00,  7.10s/it]


In [ ]:
model.y

array([-43.24878712, -56.56881882, -95.17307156, -17.73245967,
        14.37601074, -17.12083759, -42.43889565, -18.46664907,
       -71.28050327, -17.64307009,  14.52890553,  16.87358307,
        16.84290905,  16.82167402,  16.7859616 ,  16.79612293,
        16.80405684,  16.78436532,  16.8287561 ,  16.78901508,
        16.8464506 ,  16.8048295 ,  16.78706535,  16.7972334 ,
        16.80848033,  16.78779944,  16.78886658,  16.83655616,
        16.82893139,  16.82394734])

In [ ]:
stop_iter = 30
bound = np.array([[20, -20],[20, -20],[20, -20],[20, -20],[20, -20]])
param = {"a0" : 200, "a1" : 0.05, "a2" : 0, "a3" : 0}
model = bayesian_opt(bound = bound, param = param, ac = "UCB", prior_dist = True, prior_x = prior_x, prior_y = prior_y[:-1])
# x_new = model.V[0]
y_new = np.array([prior_y[-1]])
for i in tqdm(range(stop_iter)):
  x_new = model.fit(y_new)
  y_new = np.array([test_func(x_new[0])])

100%|██████████| 30/30 [01:41<00:00,  3.38s/it]


In [ ]:
model.y

array([0.09071357, 0.12349125, 0.19794354, 0.1863763 , 0.198982  ,
       0.14845109, 0.12165627, 0.17002733, 0.3195646 , 0.30308254,
       0.30956788, 0.18675442, 0.21652036, 0.35011485, 0.21671971,
       0.26567168, 0.1998899 , 0.27947313, 0.2090295 , 0.34674608,
       0.28956933, 0.2568092 , 0.4236771 , 0.24773637, 0.38058719,
       0.37181504, 0.33782685, 0.20395884, 0.25041127, 0.36629486,
       0.3297422 , 0.33702121, 0.31797976, 0.38086529, 0.36241022,
       0.26834531, 0.30286587, 0.28470936, 0.31738752, 0.37045469,
       0.25537529, 0.36523017, 0.43686212, 0.35087642, 0.30137309,
       0.39411681, 0.31848551, 0.34232397, 0.33560797, 0.30717   ,
       0.35288354, 0.26493074, 0.44786914, 0.32669078, 0.45362864,
       0.44312776, 0.44962611, 0.40875243, 0.45562591, 0.57447997,
       0.61198924, 0.54839157, 0.59865223, 0.57871177, 0.5243241 ,
       0.53472481, 0.53851168, 0.59061138, 0.67111032, 0.47047325,
       0.61481953, 0.60924359, 0.65534322, 0.59967073, 0.68813

In [ ]:
np.array([prior_y[-1]])

array([39.50311202])

In [ ]:
import numpy as np

matrix = np.array([[2,2,2],[4,4,4],[6,6,6]])
matrix = np.array([2,2,2])

vector = np.array([2,4,6])

matrix = matrix * vector
matrix

array([ 4,  8, 12])

### 動きを描画

In [ ]:
def plot_figure(model):
  color_dict = {0: 'red', 1: 'green', 2: 'blue', 3:'yellow'}
  c = ['red', 'green', 'blue', 'yellow']

  fig_dict = {
      "data": [],
      "layout": {},
      "frames": []
  }


  fig_dict["layout"]["hovermode"] = "closest"
  fig_dict["layout"]["updatemenus"] = [
      {
          "buttons": [
              {
                  "args": [None, {"frame": {"duration": 500, "redraw": False},
                                  "fromcurrent": True, "transition": {"duration": 300,
                                                                      "easing": "quadratic-in-out"}}],
                  "label": "Play",
                  "method": "animate"
              },
              {
                  "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}],
                  "label": "Pause",
                  "method": "animate"
              }
          ],
          "direction": "left",
          "pad": {"r": 10, "t": 87},
          "showactive": False,
          "type": "buttons",
          "x": 0.1,
          "xanchor": "right",
          "y": 0,
          "yanchor": "top"
      }
  ]

  sliders_dict = {
      "active": 0,
      "yanchor": "top",
      "xanchor": "left",
      "currentvalue": {
          "font": {"size": 20},
          "prefix": "Iteration:",
          "visible": True,
          "xanchor": "right"
      },
      "transition": {"duration": 300, "easing": "cubic-in-out"},
      "pad": {"b": 10, "t": 50},
      "len": 0.9,
      "x": 0.1,
      "y": 0,
      "steps": []
  }



  data_dict = {
              "x": [model.V[0,0]],
              "y": [model.y[0]],
              "mode": "markers",
              "marker": {
                  "color" : "red",
              }
          }
  fig_dict["data"].append(data_dict)
  data_dict = {
              "x": np.linspace(model.bound[0,0], model.bound[0,1], 1000),
              "y": test_func2(np.linspace(model.bound[0,0], model.bound[0,1], 1000)),
              "mode": "lines",
              "name": "f(x)", 
              "marker": {
                  "color" : "blue",
              }
          }
  fig_dict["data"].append(data_dict)
  # for i in range(k):

  #     data_dict = {
  #             "x": [mu[i, 0]],
  #             "y": [mu[i, 1]],
  #             "mode": "markers",
  #             "marker_line_color" : "midnightblue",
  #             "marker_line_width" : 2,
  #             "marker": {
  #                 "color" : c[i],
  #                 "symbol" : "star",
  #                 "size" : 10
  #             }
  #         }

  #     fig_dict["data"].append(data_dict)

  for _iter in range(model.V.shape[0]):
      # color_point = np.empty(label.shape[1],dtype=object)
      # for j in range(label.shape[1]):
      #   color_point[j] = color_dict[label[_iter,j]]


      frame = {"data": [], "name": str(_iter)}
      data_dict = {
              "x": model.V[:_iter+1,0],
              "y": model.y[:_iter+1],
              "mode": "markers",
              "marker": {
                  "color" : "red",
              }
          }
      frame["data"].append(data_dict)
      data_dict = {
              "x": np.linspace(model.bound[0,0], model.bound[0,1], 1000),
              "y": test_func2(np.linspace(model.bound[0,0], model.bound[0,1], 1000)),
              "mode": "lines",
              "name": "f(x)", 
              "marker": {
                  "color" : "blue",
              }
          }
      frame["data"].append(data_dict)
      # for i in range(K):

      #     data_dict = {
      #         "x": [mu[i, 0]],
      #         "y": [mu[i, 1]],
      #         "mode": "markers",
      #         "marker_line_color" : "midnightblue",
      #         "marker_line_width" : 2,
      #         "marker": {
      #             "color" : c[i],
      #             "symbol" : "star",
      #             "size" : 10
      #         }
      #     }
      #     frame["data"].append(data_dict)

      fig_dict["frames"].append(frame)
      slider_step = {"args": [
          [_iter],
          {"frame": {"duration": 300, "redraw": False},
          "mode": "immediate",
          "transition": {"duration": 300}}
      ],
          "label": _iter,
          "method": "animate"}
      sliders_dict["steps"].append(slider_step)
      
      # if np.abs(np.sum(diff)) < 0.0001:
      #     print('mu is converged.')
      #     break


  fig_dict["layout"]["sliders"] = [sliders_dict]

  fig = go.Figure(fig_dict)
  fig.update_layout(title='dynamics of bayesian optimization',
                  width = 800, height = 800)
  
  fig.update_layout(title=dict(text='dynamics of bayesian optimization',
                             font=dict(size=26,
                                       color='grey'),
                             y=0.88,
                            ),
                    yaxis = dict(
                          range=(np.amin(test_func2(np.linspace(model.bound[0,1], model.bound[0,0],1000))),np.amax(test_func2(np.linspace(model.bound[0,1], model.bound[0,0],1000))))
                          ),
                    xaxis = dict(
                          range=(model.bound[0,1],model.bound[0,0])
                          )
  )

  fig.show()

### plot

In [ ]:
def test_func2(x):
  return (40*np.sin(x/1.0)-np.power(0.3*(x+6.0),2)-np.power(0.2*(x-4.0),2)-1.0*np.abs(x+2.0)) / 40

In [ ]:
def test_func2(x):
  return np.exp(-1 * x**2)

In [ ]:
def test_func2(x):
  return 40.0*np.exp(-(x**2)/100)*np.sin(2*np.pi*x/10)

In [ ]:
def test_func2(x):
  return 40.0*np.exp(-((x-1)**2)/100)

In [ ]:
np.amax(test_func(model.x_sample[0,:]))

-96.28083182798841

In [ ]:
def test_func2(x):
  return np.exp(-(x-0.5)**2)*np.cos(4*np.pi*x)

In [ ]:
plot_figure(model)

In [ ]:
model.V

In [ ]:
model.y

array([0.81873075, 0.81873075, 0.81873075, 0.31080508, 0.40523008,
       0.45624361, 0.5047799 , 0.22786842, 0.25725987, 0.29817456,
       0.26488568, 0.31588691, 0.69912482, 0.98202616, 0.984584  ,
       0.92748477, 0.90795671, 0.94192256, 0.95554765, 0.97224   ,
       0.94975538, 0.92464994, 0.9200973 , 0.95656869, 0.93701833,
       0.96682906, 0.95075065, 0.95877001, 0.95927869, 0.92588757,
       0.95396383, 0.03720872, 0.91361845, 0.98454823, 0.96592335,
       0.93418457, 0.99571304, 0.9682765 , 0.94116507, 0.96045827,
       0.9437286 , 0.91836479, 0.93034238, 0.96247592, 0.95034965,
       0.94888658, 0.94407236, 0.93119053, 0.91985713, 0.94245754,
       0.96292744, 0.97727224, 0.93130616, 0.95982133, 0.94704699,
       0.93488004, 0.96522808, 0.96511673, 0.97656407, 0.95555092,
       0.94408358, 0.97409154, 0.96392868, 0.94601428, 0.97314806,
       0.97850666, 0.96079499, 0.92180296, 0.95151856, 0.95377925,
       0.93177802, 0.93717508, 0.96166475, 0.91177501, 0.94361